In [2]:
from TrajGen import (
    Movement,
    DataPoint,
    WayPoint,
)
import gpxpy
import folium
import random
from typing import Tuple, List
from folium import plugins
import matplotlib.pyplot as plt
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import uuid
import utm 
import gpxpy

center = [55.39594, 10.38831]
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [3]:
def setup_map(center, zoom_start = 14, tiles: str = "cartodbdark_matter"):
    map_ = folium.Map(
        location = center,
        zoom_start = zoom_start,
#         tiles = tiles,
    )
    plugins.Fullscreen(
        position = "topleft"
    ).add_to(map_)
    plugins.Draw(
        filename="placeholder.geojson",
        export = True,
        position = "topleft"
    ).add_to(map_)
    return map_

def plot_point(datapoint, center, color, radius = 5.0, opacity = 1, map_ = None):
    folium.CircleMarker(
        [datapoint.latitude, datapoint.longitude],
        radius = radius,
        color = color,
        opacity = opacity,
        popup = f"...",
    ).add_to(map_)
    return map_
    
def plot_polyline(datapoints, center, color, weight: float = 2.0, opacity: float = 1, map_ = None):
        lst = []
        for datapoint in datapoints:
            lst.append([
                datapoint.latitude,
                datapoint.longitude,
            ])
        folium.PolyLine(
            lst,
            color = color,
            weight = weight,
            opacity = opacity,
            popup = f"...",
        ).add_to(map_)
        return map_

def rand_24_bit() -> int:
    """Returns a random 24-bit integer"""
    return random.randrange(0, 16**6)


def color_dec() -> int:
    """Alias of rand_24 bit()"""
    return rand_24_bit()


def color_hex(num: int = rand_24_bit()) -> str:
    """Returns a 24-bit int in hex"""
    return "%06x" % num


def color_rgb(num: int = rand_24_bit()) -> Tuple[int, int, int]:
    """Returns three 8-bit numbers, one for each channel in RGB"""
    hx = color_hex(num)
    barr = bytearray.fromhex(hx)
    return (barr[0], barr[1], barr[2])


In [5]:
start_dp = DataPoint(
    latitude = 55.363451206145925,
    longitude = 10.412869042981713,
)
middle_dp = DataPoint(
    latitude = 55.373384555955425,
    longitude = 10.40937928427195,
)

end_dp = DataPoint(
    latitude = 55.37427658758254,
    longitude = 10.403394466623759,
)

waypoints = [
    WayPoint(
        latitude = 55.363451206145925,
        longitude = 10.412869042981713,
        duration = 120, # In seconds
        std = 14, # In meters
    ),
    WayPoint(
        latitude = 55.373384555955425,
        longitude = 10.40937928427195,
        duration = 120, # Seconds
        std = 14, # In meters
    ),
    WayPoint(
        latitude = 55.37427658758254,
        longitude = 10.403394466623759,
        duration = 120, # Seconds
        std = 14, # In meters
    )
]

mean_speed = 5.5                      # Average human walking speed in km/h
mean_speed = 5.5 * (1000 / (60 * 60)) # Average human walking speed in m/s
std_speed = 0.5
std_speed = 0.5 * (1000 / (60 * 60)) # Walking speed in m/s
mean_time_delta = 25.
std_time_delta = 25.

map_ = setup_map(center)
map_ = plot_point(start_dp, center, color = "red", map_ = map_)
map_ = plot_point(middle_dp, center, color = "green", map_ = map_)
map_ = plot_point(end_dp, center, color = "blue", map_ = map_)

paths = []
for _ in range(10):
    random.seed(time.time())
    color = "#" + str(color_hex(num = rand_24_bit()))
    path = Movement(
        waypoints = waypoints,
        mean_time_delta = mean_time_delta,
        std_time_delta = std_time_delta,
        mean_speed = mean_speed,
        std_speed = std_speed,
        std_datapoint = 20,
    )
    map_ = plot_polyline(
        [DataPoint(x, y) for x, y in list(path.to_latlon())],
        center,
        color = color,
        map_ = map_,
    )
    paths.append(path)
map_

In [6]:
# Start: 55.36516472074385, 10.391764905224047
# Next: 55.373222728409694, 10.395718376168764

start_latitude = 55.37510962342682
start_longitude = 10.39598961857632
middle_latitude = 55.38146629798954
middle_longitude = 10.39618251966652

end_latitude = 55.39472434817015
end_longitude = 10.393096102223307

start = DataPoint(
    latitude = start_latitude,
    longitude = start_longitude,
)
middle = DataPoint(
    latitude = middle_latitude,
    longitude = middle_longitude,
)


x, y, _, _ = utm.from_latlon(middle_latitude, middle_longitude)


init_pair = [
    WayPoint(
        latitude = start_latitude,
        longitude = start_longitude,
        duration = 240, # In seconds
        std = 14, # In meters
    ),
    WayPoint(
        latitude = middle_latitude,
        longitude = middle_longitude,
        duration = 0, # Seconds
        std = 0, # In meters
    ),
]


list_waypoints = []

for _ in range(30):
    while True:
        dx0 = np.random.normal(0, 1000)
        dx1 = np.random.normal(0, 1000)
        if np.abs(dx0) > 250 and dx1 > -250:
            break
    lat, lon = utm.to_latlon(dx0 + x, dx1 + y, 32, "U")
    list_waypoints.append(
        (1, 
            init_pair + [WayPoint(
                latitude = lat,
                longitude = lon,
                duration = 0, # Seconds
                std = 0, # In meters
            )],
        )
    )

list_waypoints.append(
    (15, 
        init_pair + [WayPoint(
            latitude = end_latitude,
            longitude = end_longitude,
            duration = 240, # In seconds
            std = 14, # In meters
        )],
    )
)

map_ = setup_map(center)
map_ = plot_point(start, center, color = "red", map_ = map_)
map_ = plot_point(middle, center, color = "green", map_ = map_)
# map_ = plot_point(end_dp, center, color = "blue", map_ = map_)

paths = []
for i, waypoints in list_waypoints:
    for _ in range(i):
        random.seed(time.time())
        color = "#" + str(color_hex(num = rand_24_bit()))
        path = Movement(
            waypoints = waypoints,
            mean_time_delta = mean_time_delta,
            std_time_delta = std_time_delta,
            mean_speed = mean_speed,
            std_speed = std_speed,
            std_datapoint = 15,
        )
        map_ = plot_polyline(
            [DataPoint(x, y) for x, y in list(path.to_latlon())],
            center,
            color = color,
            map_ = map_,
        )
        paths.append(path)
map_

In [ ]:
# gpx = gpxpy.gpx.GPX()

# for path in paths:
#     # Create first track in our GPX:
#     gpx_track = gpxpy.gpx.GPXTrack()
#     gpx.tracks.append(gpx_track)
#     # Create first segment in our GPX track:
#     gpx_segment = gpxpy.gpx.GPXTrackSegment()
#     gpx_track.segments.append(gpx_segment)
#     for lat, lon in path.to_latlon(): 
#         # Create points:
#         gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(lat, lon, time))

# with open("./tracks.gpx", "w") as f:
#     f.write(gpx.to_xml())

In [7]:
def to_dataframe(
    start_datetime: datetime,
    movements: List[Movement],
    gap = timedelta(days = 1.),
    ) -> pd.DataFrame:
    df = pd.DataFrame()
    cumulative_gap = start_datetime 
    for movement in movements:
        # Give the 'movement' a unique identifier
        identifier = str(uuid.uuid4())
        timestamps = [
            cumulative_gap + timedelta(seconds = seconds)
            for seconds in np.cumsum(movement.timestamps)
        ]
        data = [
            {"latitude": lat, "longitude": lon, "uid": identifier}
            for lat, lon in movement.to_latlon()
        ]        
        df = df.append(
            pd.DataFrame(data = data, index = timestamps)
        )
        cumulative_gap = timestamps[-1] + gap
    return df

In [8]:
def to_gpx(
    start_datetime: datetime,
    movements: List[Movement],
    gap = timedelta(days = 1.),
    filepath: str = "./tracks.gpx",
    ) -> None:
    gpx = gpxpy.gpx.GPX()
    # Create first track in our GPX:
    gpx_track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(gpx_track)
    # Create first segment in our GPX track:
    gpx_segment = gpxpy.gpx.GPXTrackSegment()
    gpx_track.segments.append(gpx_segment)
    df = to_dataframe(
        start_datetime = start_datetime,
        movements = movements,
        gap = gap,
    )
    for index, row in df.iterrows():
        # Create points:
        gpx_segment.points.append(
            gpxpy.gpx.GPXTrackPoint(
                latitude = float(row["latitude"]),
                longitude = float(row["longitude"]),
                time = index.to_pydatetime(),
            )
        )
    with open(filepath, "w") as f:
        f.write(gpx.to_xml())
    return df
df = to_gpx(start_datetime = datetime.now(), movements = paths)
df.to_json("./testdf.json", orient = "split")

In [43]:
# (df.index - df.index)
df#.index.shift(periods=1, freq="0.2T")
df.index.values[1:] - df.index.values[:-1]

array([72463483000, 26515263000, 38570539000, ..., 45887960000,
       29130307000, 63103481000], dtype='timedelta64[ns]')